In [1]:
proficiencia = {}
porc_vit = {}

In [2]:
import pandas as pd
import numpy as np
import random as rd
from math import ceil,floor,factorial
from scipy.optimize import minimize
Times = pd.read_excel('TIMES.xlsx')
Times.set_index('Número', inplace = True)
Times.fillna('NA', inplace = True)
Times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(Times.Sigla))), index=Times.index)
Times_msi = ['RNG','KZ','FW','FNC','TL','EVS','SUP','GMB','KBM','R7','KLG','DW','ASC','PGM']
Times_mundial_inicio = ['EDG','INF','DW','G2','SUP','ASC','C9','DFM','KBM','GRX','GMB','KLG','AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100']
Times_mundial_grupos = ['AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100','EDG','G2','C9','GRX']
Times_mundial_quartas = ['RNG','KT','FNC','IG','EDG','G2','C9','AFS']
ACERTOS=0
JOGOS_TOTAIS = 0

In [3]:
def arrumadados(sheet,guia,vero = False):
    arquivo = pd.ExcelFile(sheet)
    liga = pd.read_excel(arquivo,guia-1)
    if not vero:
        liga = liga[3:]
    liga.fillna(0,inplace = True)
    liga.replace(0.7,0.8, inplace = True)
    return liga

In [4]:
liga_time_reg = []
#Declarando as tabelas de jogos de cada regiao
cblol_2019_1 = arrumadados('CBLOL_BRASIL_2019.xlsx',1,True)
a = list(cblol_2019_1.Winner)
a.extend(list(cblol_2019_1.Loser))
times_cblol_2019 = set(a)
liga_time_reg.append((cblol_2019_1,times_cblol_2019,'BR'))

lec_2019_1 = arrumadados('LEC_EUROPA_2019.xlsx',1,True)
a = list(lec_2019_1.Winner)
a.extend(list(lec_2019_1.Loser))
times_lec_2019 = set(a)
liga_time_reg.append((lec_2019_1,times_lec_2019,'EU'))

lcs_2019_1 = arrumadados('LCS_USA_2019.xlsx',1,True)
lcs_2019_1.replace('100',100, inplace = True)
a = list(lcs_2019_1.Winner)
a.extend(list(lcs_2019_1.Loser))
times_lcs_2019 = set(a)
liga_time_reg.append((lcs_2019_1,times_lcs_2019,'NA'))

lck_2019_1 = arrumadados('LCK_COREIA_2019.xlsx',1,True)
a = list(lck_2019_1.Winner)
a.extend(list(lck_2019_1.Loser))
times_lck_2019 = set(a)
liga_time_reg.append((lck_2019_1,times_lck_2019,'KR'))

lpl_2019_1 = arrumadados('LPL_CHINA_2019.xlsx',1,True)
a = list(lpl_2019_1.Winner)
a.extend(list(lpl_2019_1.Loser))
times_lpl_2019 = set(a)
liga_time_reg.append((lpl_2019_1,times_lpl_2019,'CN'))

lms_2019_1 = arrumadados('LMS_TWHKMA_2019.xlsx',1,True)
a = list(lms_2019_1.Winner)
a.extend(list(lms_2019_1.Loser))
times_lms_2019 = set(a)
liga_time_reg.append((lms_2019_1,times_lms_2019,'TWHKMA'))

vcs_2019_1 = arrumadados('VCS_VIETNAM_2019.xlsx',1,True)
a = list(vcs_2019_1.Winner)
a.extend(list(vcs_2019_1.Loser))
times_vcs_2019 = set(a)
liga_time_reg.append((vcs_2019_1,times_vcs_2019,'VN'))

lcl_2019_1 = arrumadados('LCL_CIS_2019.xlsx',1,True)
a = list(lcl_2019_1.Winner)
a.extend(list(lcl_2019_1.Loser))
times_lcl_2019 = set(a)
liga_time_reg.append((lcl_2019_1,times_lcl_2019,'CIS'))

lla_2019_1 = arrumadados('LLA_LA_2019.xlsx',1,True)
a = list(lla_2019_1.Winner)
a.extend(list(lla_2019_1.Loser))
times_lla_2019 = set(a)
liga_time_reg.append((lla_2019_1,times_lla_2019,'LA'))

tcl_2019_1 = arrumadados('TCL_TURQUIA_2019.xlsx',1,True)
a = list(tcl_2019_1.Winner)
a.extend(list(tcl_2019_1.Loser))
times_tcl_2019 = set(a)
liga_time_reg.append((tcl_2019_1,times_tcl_2019,'TRK'))

ljl_2019_1 = arrumadados('LJL_JAPAN_2019.xlsx',1,True)
a = list(ljl_2019_1.Winner)
a.extend(list(ljl_2019_1.Loser))
times_ljl_2019 = set(a)
liga_time_reg.append((ljl_2019_1,times_ljl_2019,'JPN'))

opl_2019_1 = arrumadados('OPL_OCEANIA_2019.xlsx',1,True)
a = list(opl_2019_1.Winner)
a.extend(list(opl_2019_1.Loser))
times_opl_2019 = set(a)
liga_time_reg.append((opl_2019_1,times_opl_2019,'OCEANIA'))

In [5]:
def liga_usada(liga,times,sigla):
    global x0,b,bnds
    x0 = list(np.random.uniform(0.5,2.5,len(times)))
    b = (0.001,10.0)
    bnds = tuple([b]*len(times))
    return (liga,times,sigla)

def vero(x):
    y = 0
    proficiencia[sigla] = {}
    for time in times:
        proficiencia[sigla][time] = x[y]
        y+=1
    multiplicatorio = 1
    #Defino a constante de esquecimento k, obtida experimentalmente
    k = 0.075
    for game in range (0,liga.shape[0]):
        w = liga.iloc[game].Week
        pv = proficiencia[sigla][liga.iloc[game].Winner]
        pl = proficiencia[sigla][liga.iloc[game].Loser]
        
        #Diferenciando play-offs de fase regular
        if w>0:
            cte = 1 - (max(list(liga.Week)) - w)*k
        else:
            cte = 1+k
        multiplicatorio *= (pv/(pv+pl))**cte
    
    return -np.log(multiplicatorio)

def probabilidade(A,B,n):
    siglaA = list(Times[Times.Sigla==A]['Região'])[0]
    siglaB = list(Times[Times.Sigla==B]['Região'])[0]
    A_list = []
    B_list = []
    i,k = 0,floor(n/2)
    x = proficiencia[siglaA][A]/(proficiencia[siglaA][A]+proficiencia[siglaB][B])
    print("         {} vs {}".format(A,B))
    while (ceil(n/2)+i) <= n:
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) 
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
        k+=1
    if n==1:
        print('{} tem {:.2f} % de chance de vitória'.format(A,100*sum(A_list)))
        print('Enquanto {} tem {:.2f} %.'.format(B,100*sum(B_list)))
    else:    
        print('{} tem {:.2f} % de chance de vitória.Sendo:'.format(A,100*sum(A_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*A_list[b],ceil(n/2),b))
        print('Enquanto {} tem {:.2f} %. Sendo:'.format(B,100*sum(B_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*B_list[b],ceil(n/2),b))
    
def restricao1(x):
    return x[0]-1        

def restricao2(x):
    return sum(x)-5

con1 = {'type':'eq','fun':restricao1}
con2 = {'type':'eq','fun':restricao2}
cons = [con2]

In [ ]:
#Para calcular a proficiencia de uma liga apenas
liga,times,sigla = liga_usada(lpl_2019_1,times_lpl_2019,'CN')
sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia[sigla]

In [6]:
#Calculando todas as proficiencias de todos os times de todas as ligas
for trio in liga_time_reg:    
    liga,times,sigla = liga_usada(trio[0],trio[1],trio[2])
    sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)

In [64]:
#Pegando os dados de porcentagem de vitoria de todos os times
for trio in liga_time_reg:    
    liga,times,sigla = liga_usada(trio[0],trio[1],trio[2])
    for team in times:
        a = list(liga.Winner)
        a.extend(list(liga.Loser))
        porc_vit[team] = list(liga.Winner).count(team)/a.count(team)

In [65]:
## Criando um dataframe com os times ativos em 2019
listadesiglas = []
for i in range (0,len(proficiencia.keys())):
    for sigla in list(list(proficiencia.values())[i].keys()):
        listadesiglas.append(sigla)
Times_2019 = Times[Times.Sigla.isin(listadesiglas)]
del Times_2019['Força Inicial']
del Times_2019['Força atualizada']
lista2 = []

for time in Times_2019.Sigla:
    lista2.append(proficiencia[list(Times_2019[Times_2019.Sigla==time]['Região'])[0]][time])
    Times_2019.at[Times_2019[Times_2019.Sigla==time].index, 'Porcentagem de Vitorias'] = porc_vit[time]
            
Times_2019['Proficiencia'] = pd.Series(lista2, index=Times_2019.index)
Times_2019['Porcentagem de Vitorias'] = Times_2019['Porcentagem de Vitorias'].apply(lambda x:int(10000*x))
Times_2019['Porcentagem de Vitorias'] = Times_2019['Porcentagem de Vitorias'].apply(lambda x:x/100)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [67]:
Times_2019[Times_2019['Região']=='KR'].sort_values(by =['Proficiencia'], ascending = False).round(3)

,Sigla,Região,Porcentagem de Vitorias,Proficiencia
Número,,,,
4,GRF,KR,79.48,1.067
5,KZ,KR,69.23,1.028
7,SKT,KR,70.45,0.977
19,DWG,KR,59.52,0.555
18,SB,KR,63.63,0.528
6,HLE,KR,48.78,0.316
2,GEN,KR,36.36,0.196
3,AFS,KR,32.55,0.167
1,KT,KR,30.95,0.137


#### Modelo com constante de esquecimento melhor em 66,1 % dos casos

### Previsoes da Liga Coreana
####  Taxa de Acerto na Fase de Grupos: 25/32   78,13%
#### Taxa de Acerto em Play-Offs : 66,66%

In [72]:
probabilidade('SB','DWG',3) #OK +++

         SB vs DWG
SB tem 47.53 % de chance de vitória.Sendo:
23.38 % a de ser 2-0
24.15 % a de ser 2-1
Enquanto DWG tem 52.47 %. Sendo:
26.68 % a de ser 2-0
25.80 % a de ser 2-1


In [10]:
probabilidade('DWG','KZ',5) #OK +++

         DWG vs KZ
DWG tem 25.48 % de chance de vitória.Sendo:
4.76 % a de ser 3-0
9.11 % a de ser 3-1
11.61 % a de ser 3-2
Enquanto KZ tem 74.52 %. Sendo:
25.92 % a de ser 3-0
28.18 % a de ser 3-1
20.43 % a de ser 3-2


In [15]:
probabilidade('SKT','KZ',5) #N

         SKT vs KZ
SKT tem 31.22 % de chance de vitória.Sendo:
6.26 % a de ser 3-0
11.32 % a de ser 3-1
13.65 % a de ser 3-2
Enquanto KZ tem 68.78 %. Sendo:
21.93 % a de ser 3-0
26.12 % a de ser 3-1
20.73 % a de ser 3-2


In [8]:
probabilidade('SKT','GRF',5)

         SKT vs GRF
SKT tem 57.77 % de chance de vitória.Sendo:
15.89 % a de ser 3-0
21.85 % a de ser 3-1
20.03 % a de ser 3-2
Enquanto GRF tem 42.23 %. Sendo:
9.63 % a de ser 3-0
15.65 % a de ser 3-1
16.95 % a de ser 3-2


### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Chinesa
####  Taxa de Acerto na Fase de Grupos: 38/48   79,17%
#### Taxa de Acerto em Play-Offs : 50%

In [24]:
probabilidade('WE','JDG',5) #N

         WE vs JDG
WE tem 77.73 % de chance de vitória.Sendo:
28.50 % a de ser 3-0
29.24 % a de ser 3-1
19.99 % a de ser 3-2
Enquanto JDG tem 22.27 %. Sendo:
4.00 % a de ser 3-0
7.89 % a de ser 3-1
10.38 % a de ser 3-2


In [35]:
probabilidade('SDG','EDG',5) #OK +++

         SDG vs EDG
SDG tem 54.13 % de chance de vitória.Sendo:
14.23 % a de ser 3-0
20.40 % a de ser 3-1
19.50 % a de ser 3-2
Enquanto EDG tem 45.87 %. Sendo:
10.92 % a de ser 3-0
17.10 % a de ser 3-1
17.85 % a de ser 3-2


In [20]:
probabilidade('RNG','JDG',5) #N

         RNG vs JDG
RNG tem 68.36 % de chance de vitória.Sendo:
21.67 % a de ser 3-0
25.96 % a de ser 3-1
20.74 % a de ser 3-2
Enquanto JDG tem 31.64 %. Sendo:
6.37 % a de ser 3-0
11.48 % a de ser 3-1
13.79 % a de ser 3-2


In [8]:
probabilidade('TOP','SDG',5) #OK

         TOP vs SDG
TOP tem 46.05 % de chance de vitória.Sendo:
10.98 % a de ser 3-0
17.17 % a de ser 3-1
17.89 % a de ser 3-2
Enquanto SDG tem 53.95 %. Sendo:
14.15 % a de ser 3-0
20.33 % a de ser 3-1
19.47 % a de ser 3-2


In [9]:
probabilidade('JDG','FPX',5)

         JDG vs FPX
JDG tem 14.62 % de chance de vitória.Sendo:
2.36 % a de ser 3-0
5.05 % a de ser 3-1
7.20 % a de ser 3-2
Enquanto FPX tem 85.38 %. Sendo:
36.26 % a de ser 3-0
31.21 % a de ser 3-1
17.91 % a de ser 3-2


### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Norte Americana
####  Taxa de Acerto na Fase de Grupos: 23/31    74,19%
#### Taxa de Acerto em Play-Offs : 100%

In [57]:
probabilidade('FLY','GGS',5) #OK +++++

         FLY vs GGS
FLY tem 52.24 % de chance de vitória.Sendo:
13.42 % a de ser 3-0
19.64 % a de ser 3-1
19.18 % a de ser 3-2
Enquanto GGS tem 47.76 %. Sendo:
11.63 % a de ser 3-0
17.86 % a de ser 3-1
18.28 % a de ser 3-2


In [67]:
probabilidade('TSM','FOX',5) #OK +++

         TSM vs FOX
TSM tem 95.18 % de chance de vitória.Sendo:
53.80 % a de ser 3-0
30.13 % a de ser 3-1
11.25 % a de ser 3-2
Enquanto FOX tem 4.82 %. Sendo:
0.65 % a de ser 3-0
1.59 % a de ser 3-1
2.58 % a de ser 3-2


In [11]:
probabilidade('TSM','C9',5) #OK

         TSM vs C9
TSM tem 57.52 % de chance de vitória.Sendo:
15.77 % a de ser 3-0
21.75 % a de ser 3-1
20.00 % a de ser 3-2
Enquanto C9 tem 42.48 %. Sendo:
9.71 % a de ser 3-0
15.75 % a de ser 3-1
17.02 % a de ser 3-2


In [13]:
probabilidade('TL','FLY',5) #OK +++++

         TL vs FLY
TL tem 83.94 % de chance de vitória.Sendo:
34.57 % a de ser 3-0
30.92 % a de ser 3-1
18.44 % a de ser 3-2
Enquanto FLY tem 16.06 %. Sendo:
2.65 % a de ser 3-0
5.58 % a de ser 3-1
7.83 % a de ser 3-2


In [12]:
probabilidade('TSM','TL',5)

         TSM vs TL
TSM tem 53.26 % de chance de vitória.Sendo:
13.85 % a de ser 3-0
20.05 % a de ser 3-1
19.36 % a de ser 3-2
Enquanto TL tem 46.74 %. Sendo:
11.24 % a de ser 3-0
17.45 % a de ser 3-1
18.05 % a de ser 3-2


### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Europeia
#### Taxa de Acerto na Fase de Grupos: 16/26 61,54%
#### Taxa de Acerto em Play-Offs : 75%

In [14]:
probabilidade('FNC','VIT',5) #OK +++++

         FNC vs VIT
FNC tem 85.59 % de chance de vitória.Sendo:
36.51 % a de ser 3-0
31.25 % a de ser 3-1
17.83 % a de ser 3-2
Enquanto VIT tem 14.41 %. Sendo:
2.32 % a de ser 3-0
4.98 % a de ser 3-1
7.11 % a de ser 3-2


In [29]:
probabilidade('SPY','SK',5) #OK +++++

         SPY vs SK
SPY tem 68.00 % de chance de vitória.Sendo:
21.44 % a de ser 3-0
25.82 % a de ser 3-1
20.74 % a de ser 3-2
Enquanto SK tem 32.00 %. Sendo:
6.47 % a de ser 3-0
11.62 % a de ser 3-1
13.91 % a de ser 3-2


In [14]:
probabilidade('FNC','SPY',5) #OK +++++

         FNC vs SPY
FNC tem 71.32 % de chance de vitória.Sendo:
23.60 % a de ser 3-0
27.05 % a de ser 3-1
20.67 % a de ser 3-2
Enquanto SPY tem 28.68 %. Sendo:
5.57 % a de ser 3-0
10.33 % a de ser 3-1
12.77 % a de ser 3-2


In [10]:
probabilidade('OG','G2',5) #N

         OG vs G2
OG tem 24.30 % de chance de vitória.Sendo:
4.47 % a de ser 3-0
8.66 % a de ser 3-1
11.17 % a de ser 3-2
Enquanto G2 tem 75.70 %. Sendo:
26.83 % a de ser 3-0
28.58 % a de ser 3-1
20.29 % a de ser 3-2


In [10]:
probabilidade('OG','FNC',5)

         OG vs FNC
OG tem 16.45 % de chance de vitória.Sendo:
2.73 % a de ser 3-0
5.72 % a de ser 3-1
8.00 % a de ser 3-2
Enquanto FNC tem 83.55 %. Sendo:
34.14 % a de ser 3-0
30.84 % a de ser 3-1
18.57 % a de ser 3-2


### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Brasileira
#### Taxa de Acerto na Fase de Grupos: 29/36    80,55%
#### Taxa de Acerto em Play-Offs : 100%

In [12]:
probabilidade('FLA','CNB',5) #OK +++++

         FLA vs CNB
FLA tem 99.96 % de chance de vitória.Sendo:
90.18 % a de ser 3-0
9.16 % a de ser 3-1
0.62 % a de ser 3-2
Enquanto CNB tem 0.04 %. Sendo:
0.00 % a de ser 3-0
0.01 % a de ser 3-1
0.02 % a de ser 3-2


In [13]:
probabilidade('RDP','ITZ',5) #OK +++

         RDP vs ITZ
RDP tem 31.26 % de chance de vitória.Sendo:
6.27 % a de ser 3-0
11.33 % a de ser 3-1
13.66 % a de ser 3-2
Enquanto ITZ tem 68.74 %. Sendo:
21.91 % a de ser 3-0
26.10 % a de ser 3-1
20.73 % a de ser 3-2


In [11]:
probabilidade('FLA','ITZ',5)

         FLA vs ITZ
FLA tem 99.77 % de chance de vitória.Sendo:
82.07 % a de ser 3-0
15.69 % a de ser 3-1
2.00 % a de ser 3-2
Enquanto ITZ tem 0.23 %. Sendo:
0.03 % a de ser 3-0
0.07 % a de ser 3-1
0.14 % a de ser 3-2


#### PS: em outros campeonatos : 3-2 em play-offs

# Calculo da proficiencia relativa entre as regioes

In [551]:
#Criando um dicionario com todas as regioes
regions = {}
for reg in set(Times.Região): 
    regions[reg] = list(Times.Sigla[Times.Região == reg])
    
#Criando um com todos os mundiais e importando os respectivos jogos
    mundial = {}
for ano in range (2011,2019):
    mundial[ano] = arrumadados("MUNDIAIS.xlsx",ano-2010,True)

#Criando um dataframe com as regiões e suas proficiências em cada ano
Proficiencias_Relativas = pd.DataFrame(data = None, index = list(set(Times.Região)),columns = [2011,2012,2013,2014,2015,2016,2017,2018,2019])


In [553]:
#Criando uma função que adiciona a proficiência relativa de cada região no dataframe
def adicione(ano):
    m = min(proficiencia.values())
    M = max(proficiencia.values())
    for reg in set(Times.Região):
        if reg in list(proficiencia.keys()):
            Proficiencias_Relativas.loc[[reg],[ano]] = (proficiencia[reg]-m)/(M-m)
            
#Modificando a funcao original pra que se adeque ao objetivo proposto
def vero_reg(x):
    y = 0
    for time in times:
        proficiencia_reg[time] = x[y]
        y+=1
    multiplicatorio = 1
    for ano in range(2011,2019):
        cship = mundial[ano]
        coef = 1 - (2018-ano)/8
        for game in range (0,int(cship.shape[0])):
            pv = proficiencia_reg[cship.iloc[game].Winner]
            pl = proficiencia_reg[cship.iloc[game].Loser]
            multiplicatorio *= (pv/(pv+pl))**(coef)
    
    return -np.log(multiplicatorio)

def probabilidadeR(A,B):
    x = proficiencia_reg[A]/(proficiencia_reg[A]+proficiencia_reg[B])
    print("         {} vs {}".format(A,B))
    print("{} tem {:.2f} % de chance de vitória".format(A,100*x))
    print("Enquanto {} tem {:.2f} %".format(B,100*(1-x)))

In [554]:
for camp in mundial.values():
    for reg in set(Times.Região):
        for linha in range(0,camp.shape[0]):
            if camp.iloc[linha].Winner in regions[reg]:
                camp.iloc[linha].Winner = reg
            if camp.iloc[linha].Loser in regions[reg]:
                camp.iloc[linha].Loser = reg

In [532]:
proficiencia_reg = {}
cons = con1
liga,times,sigla = liga_usada(lck_2019_1,list(set(Times.Região)),'MUND')
sol = minimize(vero_reg,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia_reg

{'BR': 0.11340527773331911,
 'CIS': 0.674702043456038,
 'CN': 1.3065021478148244,
 'EU': 1.0,
 'JPN': 0.06729952276967999,
 'KR': 2.252599874444157,
 'LAN': 0.3412384251901959,
 'LAS': 0.07090566327481033,
 'NA': 0.8036978293355066,
 'OCEANIA': 0.08932550771918506,
 'SEA': 0.015643127392125797,
 'TRK': 0.2634733283402824,
 'TWHKMA': 0.455997806299471,
 'VN': 0.3427505228918947}

In [538]:
sorted(proficiencia_reg.items(), key=lambda x: x[1])

[('SEA', 0.015643127392125797),
 ('JPN', 0.06729952276967999),
 ('LAS', 0.07090566327481033),
 ('OCEANIA', 0.08932550771918506),
 ('BR', 0.11340527773331911),
 ('TRK', 0.2634733283402824),
 ('LAN', 0.3412384251901959),
 ('VN', 0.3427505228918947),
 ('TWHKMA', 0.455997806299471),
 ('CIS', 0.674702043456038),
 ('NA', 0.8036978293355066),
 ('EU', 1.0),
 ('CN', 1.3065021478148244),
 ('KR', 2.252599874444157)]

In [542]:
probabilidadeR('LAS','BR')

         LAS vs BR
LAS tem 38.47 % de chance de vitória
Enquanto BR tem 61.53 %
